In [82]:
import pandas as pd
import numpy as np
import ast

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [83]:
df = pd.read_csv(r"../output/kfold.csv")

In [84]:
# List of metric names
metric_names = ["lift",
    "f1",
    "mse",
    "accuracy",
    "recall",
    "precision",
    "roc_auc",
    "area_under_pr",
    "r2_score",
    "rmse"
]
dataset = df.copy()
# Create columns based on metric names
# Convert string representations to dictionaries
dataset["output_metrics"] = dataset["output_metrics"].apply(ast.literal_eval)
# Create columns based on metric names
for metric in metric_names:
    dataset[metric] = dataset["output_metrics"].apply(
        lambda x: x.get(metric) if isinstance(x, dict) else np.nan
    )
    dataset[f"{metric}_std"] = dataset[metric].apply(
        lambda x: np.std(x) if isinstance(x, list) else np.nan
    )
    dataset[metric] = dataset[metric].apply(
        lambda x: np.average(x) if isinstance(x, list) else np.nan
    )

base_cols   = [
    "dataset",
    "model",
    "run_time",
    "eval_metric",
    "best_params",
    "best_score",
    "score_std",
    "output_metrics"]

base_cols += [i for i in metric_names]
base_cols += [i+"_std" for i in metric_names]

dataset = dataset[base_cols]

In [96]:
tomax = {
    "mse": False,
    "rmse": False,
    "accuracy": True,
    "recall": True,
    "precision": True,
    "roc_auc": True,
    "area_under_pr": True,
    "lift": True,
    "f1": True,
    "r2_score": True,
}

# Define a function to select the first row based on whether to maximize or minimize the "best_score"
def select_first_row(group):
    metric = group["eval_metric"].iloc[0]
    ascending = not tomax.get(metric, True)  # If metric not in tomax, assume True (maximize)
    return group.sort_values(by="best_score", ascending=ascending).iloc[0]


# Apply the function to each group
filtered_df = filtered_df.groupby(["dataset", "model", "eval_metric"]).apply(select_first_row)
# Reset the index to get a new DataFrame
filtered_df = filtered_df.reset_index(drop=True)
# Find the rows that maximize the specified metric for each dataset


# Sort the DataFrame based on whether the metric is to be maximized or not
filtered_df["ascending"] = filtered_df["eval_metric"].map({k:not v for k,v in tomax.items()})  # Create a new column for ascending order
dfmax = filtered_df.loc[filtered_df["ascending"] == False].sort_values(by=["dataset", "eval_metric", "best_score"], ascending=[False, False, False])
dfmin = filtered_df.loc[filtered_df["ascending"] == True].sort_values(by=["dataset", "eval_metric", "best_score"], ascending=[False, False, True])

best_df = pd.concat([dfmax,dfmin])
best_df.drop(columns=["ascending"], inplace=True)
best_df.head(20)

,dataset,model,run_time,eval_metric,best_params,best_score,score_std,output_metrics,lift,f1,mse,accuracy,recall,precision,roc_auc,area_under_pr,r2_score,rmse,lift_std,f1_std,mse_std,accuracy_std,recall_std,precision_std,roc_auc_std,area_under_pr_std,r2_score_std,rmse_std
13,titanic,xgb,0.583486,lift,"{'alpha': 3.9316627966597033, 'colsample_bytre...",2.483185,0.124336,"{'recall': [0.6956521739130435, 0.661764705882...",2.483185,0.732687,NaN,0.813703,0.666539,0.816965,0.862084,0.832333,NaN,NaN,0.124336,0.030082,NaN,0.018823,0.041865,0.045743,0.024421,0.017240,NaN,NaN
8,titanic,mlp,5.219378,lift,"{'activation': 'logistic', 'alpha': 0.00017195...",2.207297,0.212164,"{'recall': [0.7246376811594203, 0.647058823529...",2.207297,0.679311,NaN,0.756431,0.672336,0.687374,0.732221,0.698215,NaN,NaN,0.212164,0.041073,NaN,0.030849,0.043343,0.047081,0.038267,0.038188,NaN,NaN
10,titanic,resnet,302.010760,lift,"{'AdamW_learning_rate': 0.0011723958594575317,...",2.145494,0.505266,"{'recall': [0.855072463768116, 0.8088235294117...",2.145494,0.665373,NaN,0.728423,0.713683,0.644341,0.806334,0.743390,NaN,NaN,0.505266,0.052284,NaN,0.040085,0.127959,0.070147,0.048077,0.100811,NaN,NaN
12,titanic,xgb,6.943799,accuracy,"{'alpha': 3.3470534018722558, 'colsample_bytre...",0.832754,0.028829,"{'recall': [0.7246376811594203, 0.764705882352...",NaN,0.765091,NaN,0.832754,0.710529,0.829900,NaN,0.702147,NaN,NaN,NaN,0.041891,NaN,0.028829,0.044901,0.047407,NaN,0.047687,NaN,NaN
6,titanic,catboost,64.763489,accuracy,"{'bagging_temperature': 0.9928196516799518, 'd...",0.830513,0.028147,"{'recall': [0.7246376811594203, 0.764705882352...",NaN,0.766997,NaN,0.830513,0.727962,0.811079,NaN,0.696238,NaN,NaN,NaN,0.040616,NaN,0.028147,0.043755,0.042184,NaN,0.044844,NaN,NaN
11,titanic,s1dcnn,4107.714681,accuracy,{'AdamW_learning_rate': 0.00015728323871932486...,0.805825,0.019104,"{'recall': [0.6231884057971014, 0.676470588235...",NaN,0.721057,NaN,0.805825,0.655200,0.810598,0.816381,0.661586,NaN,NaN,NaN,0.026145,NaN,0.019104,0.052985,0.066864,0.010880,0.035078,NaN,NaN
9,titanic,resnet,5209.657183,accuracy,"{'AdamW_learning_rate': 0.0018107067121085109,...",0.805825,0.021586,"{'recall': [0.6811594202898551, 0.764705882352...",NaN,0.722766,NaN,0.805825,0.669650,0.797970,0.850005,0.659132,NaN,NaN,NaN,0.049331,NaN,0.021586,0.090171,0.047494,0.022843,0.035335,NaN,NaN
7,titanic,mlp,80.996895,accuracy,"{'activation': 'relu', 'alpha': 0.000620500257...",0.804708,0.010990,"{'recall': [0.6956521739130435, 0.691176470588...",NaN,0.726242,NaN,0.804708,0.675533,0.789479,NaN,0.656769,NaN,NaN,NaN,0.014483,NaN,0.010990,0.035299,0.045488,NaN,0.022011,NaN,NaN
2,iris,mlp,4.168072,accuracy,"{'activation': 'logistic', 'alpha': 0.00018611...",0.966667,0.042164,"{'accuracy': [1.0, 1.0, 0.9333333333333333, 1....",NaN,0.966482,NaN,0.966667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.042350,NaN,0.042164,NaN,NaN,NaN,NaN,NaN,NaN
1,iris,catboost,34.557710,accuracy,"{'bagging_temperature': 0.14228166126511824, '...",0.960000,0.038873,"{'accuracy': [1.0, 0.9666666666666667, 0.93333...",NaN,0.959798,NaN,0.960000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.039040,NaN,0.038873,NaN,NaN,NaN,NaN,NaN,NaN
